In [1]:
import numpy as np
import re
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from pickle import dump
from pickle import load 
from keras.models import load_model
import pickle

Using TensorFlow backend.


In [2]:
# We train only on shakespeare data
X = load(open("X_training_shakespeare_final.h5", "rb"))
y = load(open("y_training_shakespeare_final.h5", "rb")) 
print(X.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'X_training_shakespeare_final.h5'

In [ ]:
mapping = load(open("mapping.pkl", "rb"))
vocab_size = len(mapping)

In [17]:
# We will initialize the first LSTM layer 
# and the last softmax Dense layer of
# our new double-layer LSTM model with the 
# pretrained layers from Model 4
prev_model = load_model("model4_3.h5")
prev_LSTM_layer = prev_model.layers[0]
prev_Dense_layer = prev_model.layers[1]

In [20]:
# Double-stacked LSTM architecture (Model 5)
# We added a Dropout layer and another 120 units LSTM layer 
model = Sequential()
model.add(LSTM(120, return_sequences=True, input_shape = (X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(120))
model.add(Dense(len(mapping), activation='softmax'))

In [26]:
# Initialize with pretrained weights
model.layers[0].set_weights(prev_LSTM_layer.get_weights())
model.layers[3].set_weights(prev_Dense_layer.get_weights())
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 40, 120)           86880     
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 120)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 120)               115680    
_________________________________________________________________
dense_3 (Dense)              (None, 60)                7260      
Total params: 209,820
Trainable params: 209,820
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
# Model 5 training conditions 
# Saved model every 5 epochs
# Total 30 epochs
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs = 5, batch_size = 32, verbose=1)

Epoch 1/1
91478/91478 [==============================] - 338s 4ms/step - loss: 1.6277 - acc: 0.5067
